In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.\
builder.\
config('spark.shuffle.useOldFetchProtocol','true').\
config('spark.ui.port','0').\
config("spark.sql.warehouse.dir","/user/itv012713/warehouse").\
enableHiveSupport().\
master('yarn').\
getOrCreate()



In [35]:
from pyspark.sql.functions import current_timestamp, regexp_replace, col, when, length

## Generating a Dataframe with proper datatypes

In [3]:
customers_schema = "member_id string,emp_title string,emp_length string,home_ownership string,annual_inc float,addr_state string,zip_code string,country string,grade string,sub_grade string,verification_status string,tot_hi_cred_lim float,application_type string,annual_inc_joint float,verification_status_joint string"

In [4]:
customers_raw_df = spark.read.format("csv").option("header",True).schema(customers_schema).load("/user/itv012713/lendingclubproject/raw/customers_data_csv")

In [5]:
customers_raw_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
9cb79aa7323e81be1...,Supervisor,10+ years,MORTGAGE,125000.0,KY,424xx,USA,E,E3,Verified,594540.0,Individual,null,null
0dd2bbc517e3c8f9e...,Security,< 1 year,RENT,54000.0,WI,532xx,USA,A,A4,Not Verified,57729.0,Individual,null,null
458458599d3df3bfc...,Manager,5 years,MORTGAGE,28000.0,AR,721xx,USA,C,C4,Verified,82780.0,Individual,null,null
05ea141ec28b5c7f7...,Teller,3 years,RENT,26000.0,WI,532xx,USA,A,A5,Not Verified,41114.0,Individual,null,null
aac68850fdac09fd0...,Quality Control A...,3 years,RENT,67000.0,TX,760xx,USA,F,F2,Verified,43449.0,Individual,null,null
3a423e4589e89f429...,Technician,2 years,RENT,65000.0,CA,900xx,USA,C,C3,Verified,127916.0,Joint App,118000.0,Verified
f1efcf7dfbfef21be...,Host,< 1 year,MORTGAGE,325000.0,NY,111xx,USA,B,B5,Source Verified,41000.0,Individual,null,null
c89986155a070db2e...,Guest Services Su...,2 years,MORTGAGE,60000.0,AZ,850xx,USA,C,C2,Source Verified,280890.0,Individual,null,null
118dc629b6e134419...,general manager,10+ years,MORTGAGE,74000.0,GA,310xx,USA,A,A3,Source Verified,306897.0,Individual,null,null
a86fa4b7493708333...,Sheet Metal Mechanic,9 years,MORTGAGE,54496.0,GA,307xx,USA,B,B1,Verified,153345.0,Joint App,119496.0,Verified


In [8]:
customers_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)



## Renaming columns as per business requirment

In [6]:
customers_df_renamed = customers_raw_df.withColumnRenamed("annual_inc", "annual_income") \
.withColumnRenamed("addr_state", "address_state") \
.withColumnRenamed("zip_cod", "address_zipcode" ) \
.withColumnRenamed("country", "address_country" ) \
.withColumnRenamed("tot_hi_cred_lim", "total_high_credit_limit" ) \
.withColumnRenamed("annual_inc_joint", "join_annual_income" )

In [7]:
customers_df_renamed

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint
9cb79aa7323e81be1...,Supervisor,10+ years,MORTGAGE,125000.0,KY,424xx,USA,E,E3,Verified,594540.0,Individual,null,null
0dd2bbc517e3c8f9e...,Security,< 1 year,RENT,54000.0,WI,532xx,USA,A,A4,Not Verified,57729.0,Individual,null,null
458458599d3df3bfc...,Manager,5 years,MORTGAGE,28000.0,AR,721xx,USA,C,C4,Verified,82780.0,Individual,null,null
05ea141ec28b5c7f7...,Teller,3 years,RENT,26000.0,WI,532xx,USA,A,A5,Not Verified,41114.0,Individual,null,null
aac68850fdac09fd0...,Quality Control A...,3 years,RENT,67000.0,TX,760xx,USA,F,F2,Verified,43449.0,Individual,null,null
3a423e4589e89f429...,Technician,2 years,RENT,65000.0,CA,900xx,USA,C,C3,Verified,127916.0,Joint App,118000.0,Verified
f1efcf7dfbfef21be...,Host,< 1 year,MORTGAGE,325000.0,NY,111xx,USA,B,B5,Source Verified,41000.0,Individual,null,null
c89986155a070db2e...,Guest Services Su...,2 years,MORTGAGE,60000.0,AZ,850xx,USA,C,C2,Source Verified,280890.0,Individual,null,null
118dc629b6e134419...,general manager,10+ years,MORTGAGE,74000.0,GA,310xx,USA,A,A3,Source Verified,306897.0,Individual,null,null
a86fa4b7493708333...,Sheet Metal Mechanic,9 years,MORTGAGE,54496.0,GA,307xx,USA,B,B1,Verified,153345.0,Joint App,119496.0,Verified


## Inserting Ingestion Date for data processing

In [8]:
customers_df_ingest_date = customers_df_renamed.withColumn("ingest_date", current_timestamp())

In [9]:
customers_df_ingest_date

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
9cb79aa7323e81be1...,Supervisor,10+ years,MORTGAGE,125000.0,KY,424xx,USA,E,E3,Verified,594540.0,Individual,null,null,2024-08-10 11:35:...
0dd2bbc517e3c8f9e...,Security,< 1 year,RENT,54000.0,WI,532xx,USA,A,A4,Not Verified,57729.0,Individual,null,null,2024-08-10 11:35:...
458458599d3df3bfc...,Manager,5 years,MORTGAGE,28000.0,AR,721xx,USA,C,C4,Verified,82780.0,Individual,null,null,2024-08-10 11:35:...
05ea141ec28b5c7f7...,Teller,3 years,RENT,26000.0,WI,532xx,USA,A,A5,Not Verified,41114.0,Individual,null,null,2024-08-10 11:35:...
aac68850fdac09fd0...,Quality Control A...,3 years,RENT,67000.0,TX,760xx,USA,F,F2,Verified,43449.0,Individual,null,null,2024-08-10 11:35:...
3a423e4589e89f429...,Technician,2 years,RENT,65000.0,CA,900xx,USA,C,C3,Verified,127916.0,Joint App,118000.0,Verified,2024-08-10 11:35:...
f1efcf7dfbfef21be...,Host,< 1 year,MORTGAGE,325000.0,NY,111xx,USA,B,B5,Source Verified,41000.0,Individual,null,null,2024-08-10 11:35:...
c89986155a070db2e...,Guest Services Su...,2 years,MORTGAGE,60000.0,AZ,850xx,USA,C,C2,Source Verified,280890.0,Individual,null,null,2024-08-10 11:35:...
118dc629b6e134419...,general manager,10+ years,MORTGAGE,74000.0,GA,310xx,USA,A,A3,Source Verified,306897.0,Individual,null,null,2024-08-10 11:35:...
a86fa4b7493708333...,Sheet Metal Mechanic,9 years,MORTGAGE,54496.0,GA,307xx,USA,B,B1,Verified,153345.0,Joint App,119496.0,Verified,2024-08-10 11:35:...


## Removing duplicate Rows

In [10]:
customers_df_ingest_date.count()

2260701

In [11]:
customers_distinct = customers_df_ingest_date.distinct()

In [12]:
customers_distinct.count()

2260638

In [13]:
customers_distinct.createOrReplaceTempView("customers")

In [14]:
spark.sql("select * from customers")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
4eeed44617131f494...,casino royale,1 year,RENT,36000.0,CA,946xx,USA,C,C1,Not Verified,null,Individual,null,null,2024-08-10 11:35:...
7d1266398e6b4459e...,"Sonix, Inc.",10+ years,MORTGAGE,220000.0,CA,926xx,USA,C,C2,Verified,null,Individual,null,null,2024-08-10 11:35:...
aa482bda797428cdf...,null,5 years,RENT,100000.0,CA,925xx,USA,A,A3,Not Verified,null,Individual,null,null,2024-08-10 11:35:...
d2b7b3d7cf5a1581f...,US Army,9 years,RENT,58116.0,FL,328xx,USA,D,D4,Source Verified,null,Individual,null,null,2024-08-10 11:35:...
07cab3d75a364f0ad...,Sacramento County,7 years,MORTGAGE,85000.0,CA,957xx,USA,B,B3,Source Verified,null,Individual,null,null,2024-08-10 11:35:...
acf07791be6af87c8...,Machinist,10+ years,MORTGAGE,62000.0,NY,147xx,USA,C,C2,Source Verified,37261.0,Individual,null,null,2024-08-10 11:35:...
f367c16027d68f378...,lieutenant,10+ years,MORTGAGE,100000.0,OK,731xx,USA,C,C1,Source Verified,122485.0,Individual,null,null,2024-08-10 11:35:...
a7f6dfe1f6a47a514...,Legal Assistant,< 1 year,MORTGAGE,24960.0,VA,240xx,USA,D,D3,Verified,45049.0,Individual,null,null,2024-08-10 11:35:...
8621dad86cecff25f...,Curriculum Developer,9 years,MORTGAGE,60000.0,NJ,080xx,USA,D,D4,Verified,270045.0,Individual,null,null,2024-08-10 11:35:...
2f112d41b77db07fc...,Co Manager,10+ years,RENT,70000.0,FL,712xx,USA,C,C5,Source Verified,13900.0,Individual,null,null,2024-08-10 11:35:...


## Removing rows where annual income is null

In [15]:
spark.sql("select count(*) from customers where annual_income is null")

count(1)
5


In [17]:
customers_income_filtered = spark.sql("select * from customers where annual_income is not null")

In [18]:
customers_income_filtered

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
9257d8086c1ab441d...,transit operator,10+ years,MORTGAGE,120000.0,CA,945xx,USA,B,B5,Source Verified,540904.0,Individual,null,null,2024-08-10 11:36:...
5f853b5865a967372...,Assistant Directo...,1 year,RENT,50000.0,NY,101xx,USA,A,A2,Not Verified,40000.0,Individual,null,null,2024-08-10 11:36:...
e8a70bc677730e5c0...,Table Games Dealer,2 years,RENT,135000.0,MD,210xx,USA,B,B4,Source Verified,192431.0,Individual,null,null,2024-08-10 11:36:...
a988346674c252adb...,RN Care Manager,1 year,MORTGAGE,75000.0,TX,783xx,USA,B,B2,Source Verified,245509.0,Individual,null,null,2024-08-10 11:36:...
a25925fcfcef0b8a7...,Vice President,9 years,MORTGAGE,72000.0,IL,627xx,USA,D,D1,Not Verified,338337.0,Individual,null,null,2024-08-10 11:36:...
79beadbca463b206d...,Customer Interfac...,10+ years,MORTGAGE,70000.0,TX,774xx,USA,A,A4,Not Verified,228537.0,Individual,null,null,2024-08-10 11:36:...
d2775dd5e2e8e704f...,null,10+ years,MORTGAGE,56000.0,NM,875xx,USA,B,B2,Source Verified,32895.0,Individual,null,null,2024-08-10 11:36:...
5c50e86e02ccf4d51...,Sr. Account Execu...,< 1 year,MORTGAGE,375000.0,CA,946xx,USA,A,A1,Verified,930874.0,Individual,null,null,2024-08-10 11:36:...
81122aa25bc4083a3...,Teacher,2 years,MORTGAGE,58000.0,TX,761xx,USA,C,C5,Source Verified,279078.0,Individual,null,null,2024-08-10 11:36:...
eb0d59ffc6af6d91a...,Sr OPS Specialist II,5 years,MORTGAGE,31728.0,TX,782xx,USA,D,D2,Verified,258904.0,Joint App,91528.0,Verified,2024-08-10 11:36:...


## Convert emp_length to an integer format to enable performing operations in integer steps.

In [19]:
customers_income_filtered.createOrReplaceTempView("customers")

In [20]:
spark.sql("select distinct(emp_length) from customers")

emp_length
9 years
5 years
null
1 year
2 years
7 years
8 years
4 years
6 years
3 years


In [21]:
customers_emplength_clean = customers_income_filtered.withColumn("emp_length", regexp_replace(col("emp_length"), "(\D)",""))

In [23]:
customers_emplength_clean

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
2c581585139aba3dd...,Business Management,7,RENT,92000.0,NY,100xx,USA,B,B2,Not Verified,32500.0,Individual,null,null,2024-08-10 11:36:...
52b28e40efcf66b85...,General Manager,1,MORTGAGE,57500.0,OH,445xx,USA,C,C1,Not Verified,273596.0,Joint App,103500.0,Not Verified,2024-08-10 11:36:...
61e5b62b29898efcc...,Program Manager,1,RENT,40000.0,CA,906xx,USA,A,A5,Source Verified,33400.0,Joint App,70000.0,Source Verified,2024-08-10 11:36:...
06e7969255b74de35...,National Sales As...,10,OWN,36000.0,FL,328xx,USA,C,C4,Source Verified,36301.0,Individual,null,null,2024-08-10 11:36:...
0dc855834741e8e57...,General Manager,5,RENT,55000.0,NJ,077xx,USA,B,B3,Not Verified,54262.0,Individual,null,null,2024-08-10 11:36:...
6b032b1955bc41b75...,Marketing Associate,1,MORTGAGE,60000.0,KS,667xx,USA,C,C5,Verified,239161.0,Joint App,132000.0,Verified,2024-08-10 11:36:...
f71983c40e1f686cd...,null,null,MORTGAGE,41000.0,SC,299xx,USA,C,C4,Verified,71988.0,Individual,null,null,2024-08-10 11:36:...
efd6a896f3fd1663e...,OFFICE DEPUTY,10,MORTGAGE,38900.0,NC,286xx,USA,C,C5,Source Verified,16600.0,Joint App,77380.0,Source Verified,2024-08-10 11:36:...
df0ebef9305bdc3c7...,Sr. Staff Engineer,10,MORTGAGE,111500.0,RI,028xx,USA,C,C4,Source Verified,512457.0,Individual,null,null,2024-08-10 11:36:...
c9d893348cd43dbb1...,manager,1,MORTGAGE,150000.0,FL,347xx,USA,D,D2,Not Verified,312995.0,Joint App,250000.0,Not Verified,2024-08-10 11:36:...


In [32]:
customers_emplength_clean.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- total_high_credit_limit: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [24]:
customers_emplength_casted = customers_emplength_clean.withColumn("emp_length", customers_emplength_clean.emp_length.cast('int'))

In [35]:
customers_emplength_casted.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- total_high_credit_limit: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



## Replacing all the nulls in the emp_length with average of that column

In [25]:
customers_emplength_casted.filter("emp_length is null").count()

146903

In [26]:
customers_emplength_casted.createOrReplaceTempView("customers")

In [27]:
avg_emp_length = spark.sql("select floor(avg(emp_length)) as avg_emplength from customers").collect()

In [28]:
avg_emp_duration = avg_emp_length[0][0]

In [29]:
customers_emplength_replaced = customers_emplength_casted.na.fill(avg_emp_duration, subset = ['emp_length'])

In [30]:
customers_emplength_replaced

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
08e881d7d87001ee9...,Duquesne University,1,RENT,25855.2,PA,152xx,USA,B,B3,Source Verified,null,Individual,null,null,2024-08-10 11:37:...
d4b8ab477c6594a46...,Phoenix American,10,MORTGAGE,50062.74,FL,330xx,USA,B,B5,Verified,null,Individual,null,null,2024-08-10 11:37:...
baa80f4b69b13f51d...,Rockland NY Medic...,2,RENT,60000.0,NY,109xx,USA,C,C4,Not Verified,null,Individual,null,null,2024-08-10 11:37:...
c59019c41d7945c37...,TODD WENZEL CHEVR...,10,MORTGAGE,96600.0,MI,493xx,USA,C,C1,Verified,null,Individual,null,null,2024-08-10 11:37:...
a670c18c8dd2e05c4...,PAYLESS SHOE SOURCE,10,RENT,60000.0,NY,113xx,USA,A,A5,Not Verified,null,Individual,null,null,2024-08-10 11:37:...
9f753267d0d34485d...,State Farm,1,RENT,27000.0,CA,933xx,USA,C,C1,Verified,null,Individual,null,null,2024-08-10 11:37:...
714163185928c0c7a...,NYC Department of...,10,RENT,75500.0,NY,113xx,USA,C,C2,Verified,null,Individual,null,null,2024-08-10 11:37:...
1edd408274fe9f5af...,Stein5 LLC,1,MORTGAGE,40000.0,NC,286xx,USA,E,E3,Verified,null,Individual,null,null,2024-08-10 11:37:...
f8263ea709c7f477c...,U.S. ARMY,10,RENT,38307.6,WA,983xx,USA,D,D4,Verified,null,Individual,null,null,2024-08-10 11:37:...
9c6a9d08ea5a203ed...,null,6,RENT,30000.0,MD,208xx,USA,A,A1,Source Verified,null,Individual,null,null,2024-08-10 11:37:...


In [31]:
customers_emplength_replaced.filter("emp_length is null").count()

0

## Cleaning the column address_state

In [32]:
customers_emplength_replaced.createOrReplaceTempView("customers")

In [33]:
spark.sql("select distinct(address_state) from customers")

address_state
Helping Kenya's D...
175 (total projec...
223xx
AZ
SC
I am 56 yrs. old ...
"so Plan """"C"""" is ..."
financially I mad...
but no one will l...
LA


In [34]:
spark.sql("select count(address_state) from customers where length(address_state) > 2")

count(address_state)
254


In [37]:
customers_state_cleaned = customers_emplength_replaced.withColumn(
    "address_state",
    when(length(col("address_state")) > 2, "NA").otherwise(col("address_state"))
)

In [38]:
customers_state_cleaned

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,zip_code,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
9aeca2e75d537c2cf...,null,6,MORTGAGE,85000.0,NC,283xx,USA,C,C2,Verified,209717.0,Individual,null,null,2024-08-10 11:46:...
9e87986e2dbcca9b0...,Head Sushi Chef,3,RENT,95000.0,CA,926xx,USA,B,B1,Source Verified,56922.0,Individual,null,null,2024-08-10 11:46:...
7cdd9cc4cd6e3643d...,Administrative As...,5,MORTGAGE,35500.0,MO,654xx,USA,C,C1,Verified,151546.0,Individual,null,null,2024-08-10 11:46:...
3be6c6c8d242f2acf...,nurse,9,RENT,64255.0,NJ,076xx,USA,B,B4,Source Verified,71221.0,Individual,null,null,2024-08-10 11:46:...
b275eefd4f558165b...,General manager,10,MORTGAGE,75000.0,IL,612xx,USA,A,A1,Not Verified,203110.0,Individual,null,null,2024-08-10 11:46:...
c2844610743c6e127...,Trader/CSR,10,MORTGAGE,72000.0,NV,894xx,USA,A,A2,Source Verified,164262.0,Individual,null,null,2024-08-10 11:46:...
51464a0ff83587565...,General Manager,6,MORTGAGE,50000.0,WV,254xx,USA,A,A1,Not Verified,197049.0,Individual,null,null,2024-08-10 11:46:...
fb1305d7b392c344c...,LPN,1,OWN,40000.0,FL,344xx,USA,A,A3,Source Verified,48066.0,Individual,null,null,2024-08-10 11:46:...
f56844281329a3888...,Finance Manager,1,RENT,100000.0,NY,100xx,USA,A,A1,Source Verified,83107.0,Individual,null,null,2024-08-10 11:46:...
ed2a881b04878ae26...,Labor Relations S...,10,MORTGAGE,165000.0,NY,105xx,USA,B,B4,Not Verified,864505.0,Individual,null,null,2024-08-10 11:46:...


In [39]:
customers_state_cleaned.select("address_state").distinct()

address_state
AZ
SC
LA
MN
NJ
DC
OR
NA
VA
null


## Writing the data back to hdfs in parquet and csv format for further use

In [41]:
customers_state_cleaned.write.format("parquet").mode("overwrite").option("path", "/user/itv012713/lendingclubproject/clean_data/customers_cleaned").save()

In [42]:
customers_state_cleaned.write.option("header",True).format("csv").mode("overwrite").option("path", "/user/itv012713/lendingclubproject/clean_data/customers_cleaned_csv").save()